# Part I. One-sided finite differences

Write a function, `deriv`, which computes a derivative of its argument at a given point, $x$, using a one-sided finite difference rule with a given step side $h$, with the approximation order of $O(h^2)$. 

In [3]:
def deriv(f, x, h):
    """ Compute a derivative of `f` at point `x` with step size `h`.
    
    Compute the derivative using the one-sided rule of the approximation order of $O(h^2)$.
    
    Parameters
    ----------
    f : callable
        The function to differentiate
    x : float
        The point to compute the derivative at.
    h : float
        The step size for the finite different rule.
        
    Returns
    -------
    fder : derivative of f(x) at point x using the step size h.
    """
    return (-1.5*f(x) + 2*f(x+h) - 1/2*f(x+2*h))/h

#### Test I.1

Test your function on a simple test case: differentiate $f(x) = x^3$ at $x=0$. Comment on whether your results are consistent with the expected value of $f'(x) = 0$ and on an expected scaling with $h\to 0$.

 (10% of the total grade)

In [9]:
x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(lambda x: x**3, x, h)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.0002
0.001000 --  -2e-06
0.000100 --  -2e-08
0.000010 --  -2e-10


The error is $O(h^2)$.\
For a scheme with the approxiamtion order of $O(h^2)$ the optimal step $h$ is about $(10^{-16})^{1/3} > 10^{-5}$. The results are consistent with that: the error is the smallest when $h = 10^{-5}$

### Test I.2

Now use a slightly more complicated function, $f(x) = x^2 \log{x}$, evaluate the derivative at $x=1$ using your one-sided rule and a two-point one-sided rule. Roughly estimate the value of $h$ where the error stops decreasing, for these two schemes. 
(15% of the total grade)

In [5]:
from math import log

def f(x):
    return x**2 * log(x)
    
def fder(x):
    return x * (2.*log(x) + 1)

In [10]:
def two_p_1_sided(f, x, h):
    return (f(x+h) - f(x))/h

In [37]:
x = 1
h_list = [pow(10, -i) for i in range(1, 11)]

for h in h_list:
    a = abs(fder(x) - two_p_1_sided(f, x, h))
    b = abs(fder(x) - deriv(f, x, h))
    print("h: %.4g -- 2 point 1-sided err: %7.4g -- my 1-sided %.4g" % (h, a, b))

h: 0.1 -- 2 point 1-sided err:  0.1533 -- my 1-sided 0.006209
h: 0.01 -- 2 point 1-sided err: 0.01503 -- my 1-sided 6.617e-05
h: 0.001 -- 2 point 1-sided err:  0.0015 -- my 1-sided 6.662e-07
h: 0.0001 -- 2 point 1-sided err: 0.00015 -- my 1-sided 6.666e-09
h: 1e-05 -- 2 point 1-sided err: 1.5e-05 -- my 1-sided 4.901e-11
h: 1e-06 -- 2 point 1-sided err: 1.5e-06 -- my 1-sided 1.94e-10
h: 1e-07 -- 2 point 1-sided err: 1.506e-07 -- my 1-sided 1.694e-09
h: 1e-08 -- 2 point 1-sided err: 8.923e-09 -- my 1-sided 1.718e-08
h: 1e-09 -- 2 point 1-sided err: 8.424e-08 -- my 1-sided 1.938e-07
h: 1e-10 -- 2 point 1-sided err: 8.289e-08 -- my 1-sided 8.274e-08


The error stops decreasing at roughly $h = 10^{-8}$ for 2-point 1-sided method and at $h = 10^{-5}$ for 3-points 1-sided scheme

### Test I.3 

Now try differentiating $x^2 \log(x)$ at $x=0$. Use the three-point one-sided rule. Note that to evaluate the function at zero, you need to special-case this value. Check the scaling of the error with $h$, explain your results. 
(25% of the total grade)

In [38]:
def f(x):
    if x == 0:
        # the limit of $x^2 log(x)$ at $x-> 0$ is zero, even though log(x) is undefined at x=0
        return 0.0
    else:
        return x**2 * log(x)
    
def fder(x):
    if x == 0:
        return 0.0
    else:
        return x*(2*log(x) + 1)

x = 0
for h in [1e-2, 1e-3, 1e-4, 1e-5]:
    err = deriv(f, x, h) - fder(x)
    print("%5f -- %7.4g" % (h, err))

0.010000 -- -0.01386
0.001000 -- -0.001386
0.000100 -- -0.0001386
0.000010 -- -1.386e-05


As expected, the error is $O(h^2)$

# Part II. Midpoint rule 

Write a function which computes a definite integral using the midpoint rule up to a given error, $\epsilon$. Estimate the error by comparing the estimates of the integral at $N$ and $2N$ elementary intervals. 

In [45]:
import numpy as np

In [96]:
def midpoint_rule(func, a, b, eps):
    """ Calculate the integral of f from a to b using the midpoint rule.
    
    Parameters
    ----------
    func : callable
        The function to integrate.
    a : float
        The lower limit of integration.
    b : float
        The upper limit of integration.
    eps : float
        The target accuracy of the estimate.
        
    Returns
    -------
    integral : float
        The estimate of $\int_a^b f(x) dx$.
    """
    def quadr_sum(f, a, b, N):
        x = np.linspace(a, b, N)
        mid_points = (x[1:] + x[:-1])/2
        return np.sum(f(mid_points))*(x[1] - x[0])
    
    N = 5
    I_n = quadr_sum(func, a, b, N)
    I_2n = eps*10
    while abs(I_2n - I_n) > eps and N < 1e5:
        I_n = I_2n
        N *= 2
        I_2n = quadr_sum(func, a, b, N)
        print("Intervals: {}, Error: {}".format(N, abs(I_2n - I_n)))
        
    return I_2n

### Test II.1

Test your midpoint rule on a simple integral, which you can calculate by paper and pencil.

Compare the rate of convergence to the expected $O(N^{-2})$ scaling by studying the number of intervals required for a given accuracy $\epsilon$.

Compare the numerical results to the value you calculated by hand. Does the deviation agree with your estimate of the numerical error?
(20% of the total grade)


In [97]:
f = lambda x: x**2

In [98]:
1/3 - midpoint_rule(f, 0, 1, eps=1e-7)

Intervals: 10, Error: 0.3323035267489712
Intervals: 20, Error: 0.0007979663258210201
Intervals: 40, Error: 0.0001760517422141894
Intervals: 80, Error: 4.14359553057575e-05
Intervals: 160, Error: 1.005627791006436e-05
Intervals: 320, Error: 2.477370823206204e-06
Intervals: 640, Error: 6.148244909853773e-07
Intervals: 1280, Error: 1.5314560064538796e-07
Intervals: 2560, Error: 3.821659888547657e-08


1.2725597409168898e-08

### Test II.2

Now use your midpoint rule to compute the value of

$$
\int_0^1\! \frac{\sin{\sqrt{x}}}{x}\, dx
$$

up to a predefined accuracy of $\epsilon=10^{-4}$.

Note that the integral contains an integrable singularity at the lower limit. Do calculations two ways: first, do a straightforward computation; next, subtract the singularity. Compare the number of iterations required to achieve the accuracy of $\epsilon$.

(30% of the total grade)

In [100]:
g = lambda x: np.sin(np.sqrt(x))/x

In [104]:
midpoint_rule(g, 0, 1, 1e-4)

Intervals: 10, Error: 1.6894515015604419
Intervals: 20, Error: 0.06288485091200768
Intervals: 40, Error: 0.04194248308726989
Intervals: 80, Error: 0.02882025487535289
Intervals: 160, Error: 0.020091347902667644
Intervals: 320, Error: 0.014106333229680024
Intervals: 640, Error: 0.009939378727305659
Intervals: 1280, Error: 0.007015743719722689
Intervals: 2560, Error: 0.0049564770796921565
Intervals: 5120, Error: 0.003503201622899077
Intervals: 10240, Error: 0.002476586965677763
Intervals: 20480, Error: 0.0017510166777789937
Intervals: 40960, Error: 0.0012380868865000672
Intervals: 81920, Error: 0.0008754352739643867
Intervals: 163840, Error: 0.0006190176047349283


1.8906717161256956